In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import pickle
import os

In [3]:
# Expand the amount of data we see when showing data frames
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Getting data from my SQL database

First I want to use Pandas to get some data from my MySQL database.

In [4]:
# format: engine = create_engine('mysql+pymysql://<user>:<password>@<host>[:<port>]/<dbname>')
db_user = os.environ.get('MYSQL_USER')
db_pass = os.environ.get('MYSQL_PASS')
db_host = os.environ.get('MYSQL_HOST')
engine = create_engine(f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/subways")

In [5]:
df = pd.read_sql_query("""
    SELECT * 
    FROM traintimes 
    WHERE
        (snapshotNYC BETWEEN "2019-01-22" AND "2019-01-24") 
        AND routeId="1" 
        AND trainOrderLine=0 
    ORDER BY 
        direction, stationIdGTFS, snapshotNYC;
""", engine)

In [9]:
len(df)

213477

In [10]:
# saving what we have so far as a pickle file

pickle.dump(df, open('saved_sample2.p', 'wb'))

In [6]:
df = pickle.load(open('saved_sample2.p', 'rb'))

## Side Trip: Play with reshaping

I want to make each row one datetime with columns that comprise `route_station_dirction` like `1_103_N`. And, actually I want to go further, with `1_103_N_nexttrain` and `1_103_N_lasttrain` and maybe `1_103_N_train0to1gap`.

So first seeing if I can do that with the raw data I have. Using [this documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html), particularly the part about multiple variables, as a reference. So turning:

```python
>>> df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
...                            'two'],
...                    'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
...                    'baz': [1, 2, 3, 4, 5, 6],
...                    'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
>>> df
```
```
    foo   bar  baz  zoo
0   one   A    1    x
1   one   B    2    y
2   one   C    3    z
3   two   A    4    q
4   two   B    5    w
5   two   C    6    t
```

... into ...

```python
>>> df.pivot(index='foo', columns='bar', values=['baz', 'zoo'])
```
```
      baz       zoo
bar   A  B  C   A  B  C
foo
one   1  2  3   x  y  z
two   4  5  6   q  w  t
```

Then will look at how to flatten the columns into, essentially, `baz_A`, `bas_B` ... `zoo_C`.



First I add a `name` column to the data frame:

In [7]:
### using "zip" as described in the Fast.AI class, which Jeremy says is 300x faster
underscore="_"
result = []

for r,s,d in zip(df.routeId.values, df.stationIdGTFS.values, df.direction.values):
    result.append(r+underscore+s+underscore+d)
df['name'] = result

In [8]:
### This runs slower, but would do the same thing
# df['name']=df.apply(lambda row: f'{row.routeId}_{row.stationIdGTFS}_{row.direction}', axis=1)

In [10]:
# there it is ...
df

,id,snapshotUnix,snapshotNYC,routeId,stationIdGTFS,direction,trainOrderLine,trainOrderAll,arrivalTime,departureTime,updatedOn,timeToArrival,timeToDeparture,delay,name
0,128769,1548133274,2019-01-22 00:01:14,1,101,N,0,0,1.548134e+09,NaN,1548133254,358.0,NaN,None,1_101_N
1,133212,1548133334,2019-01-22 00:02:14,1,101,N,0,0,1.548134e+09,NaN,1548133329,232.0,NaN,None,1_101_N
2,137594,1548133394,2019-01-22 00:03:14,1,101,N,0,0,1.548134e+09,NaN,1548133374,187.0,NaN,None,1_101_N
3,145548,1548133514,2019-01-22 00:05:14,1,101,N,0,0,1.548134e+09,NaN,1548133494,22.0,NaN,None,1_101_N
4,149848,1548133574,2019-01-22 00:06:14,1,101,N,0,0,1.548134e+09,NaN,1548133569,-49.0,NaN,None,1_101_N
5,154190,1548133634,2019-01-22 00:07:14,1,101,N,0,0,1.548134e+09,NaN,1548133629,654.0,NaN,None,1_101_N
6,158503,1548133694,2019-01-22 00:08:14,1,101,N,0,0,1.548134e+09,NaN,1548133674,609.0,NaN,None,1_101_N
7,162820,1548133754,2019-01-22 00:09:14,1,101,N,0,0,1.548134e+09,NaN,1548133734,557.0,NaN,None,1_101_N
8,167092,1548133814,2019-01-22 00:10:14,1,101,N,0,0,1.548134e+09,NaN,1548133794,495.0,NaN,None,1_101_N
9,171316,1548133874,2019-01-22 00:11:14,1,101,N,0,0,1.548134e+09,NaN,1548133854,457.0,NaN,None,1_101_N


In [11]:
pickle.dump(df, open('saved_sample3.p', 'wb'))

OK, now let's make a really wide table with `timeToArrival` as the value for each name.

In [25]:
df_flat = df.pivot(index='snapshotNYC', columns='name', values='timeToArrival')

In [26]:
df_flat

name,1_101_N,1_101_S,1_103_N,1_103_S,1_104_N,1_104_S,1_106_N,1_106_S,1_107_N,1_107_S,...,1_136_N,1_136_S,1_137_N,1_137_S,1_138_N,1_138_S,1_139_N,1_139_S,1_142_N,1_142_S
snapshotNYC,,,,,,,,,,,,,,,,,,,,,
2019-01-22 00:00:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247.0,256.0,157.0,316.0,97.0,526.0,37.0,616.0,NaN,108.0
2019-01-22 00:01:14,358.0,NaN,58.0,426.0,621.0,516.0,531.0,606.0,441.0,696.0,...,193.0,173.0,103.0,233.0,43.0,443.0,636.0,533.0,NaN,-2.0
2019-01-22 00:02:14,232.0,NaN,-68.0,351.0,518.0,441.0,428.0,531.0,338.0,621.0,...,123.0,60.0,33.0,120.0,621.0,330.0,561.0,420.0,NaN,540.0
2019-01-22 00:03:14,187.0,NaN,-113.0,306.0,494.0,396.0,404.0,486.0,314.0,576.0,...,79.0,13.0,624.0,73.0,564.0,283.0,504.0,373.0,NaN,493.0
2019-01-22 00:04:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.0,393.0,576.0,43.0,516.0,253.0,456.0,343.0,NaN,463.0
2019-01-22 00:05:14,22.0,NaN,486.0,186.0,396.0,276.0,306.0,366.0,216.0,456.0,...,606.0,333.0,516.0,-46.0,456.0,164.0,396.0,254.0,NaN,374.0
2019-01-22 00:06:14,-49.0,NaN,425.0,111.0,335.0,201.0,245.0,291.0,155.0,381.0,...,546.0,273.0,456.0,333.0,396.0,6.0,336.0,96.0,NaN,216.0
2019-01-22 00:07:14,654.0,NaN,354.0,51.0,264.0,141.0,174.0,231.0,84.0,321.0,...,471.0,613.0,381.0,673.0,321.0,883.0,261.0,10.0,NaN,130.0
2019-01-22 00:08:14,609.0,NaN,309.0,6.0,219.0,96.0,129.0,186.0,39.0,276.0,...,426.0,590.0,336.0,650.0,276.0,860.0,216.0,-35.0,NaN,85.0


In [27]:
len(df_flat)

2855

Can we do it with two values? (Interestingly, the example in the Pandas docs create an error. Googling around found [this solution](https://stackoverflow.com/questions/54186165/pandas-pivot-error-exception-data-must-be-1-dimensional), which works).

In [63]:
df2 = df.pivot(index='snapshotNYC', columns='name')[['timeToArrival','timeToDeparture']]

In [64]:
df2

timeToArrival                                          \
name                      1_101_N 1_101_S 1_103_N 1_103_S 1_104_N 1_104_S   
snapshotNYC                                                                 
2019-01-22 00:00:14           NaN     NaN     NaN     NaN     NaN     NaN   
2019-01-22 00:01:14         358.0     NaN    58.0   426.0   621.0   516.0   
2019-01-22 00:02:14         232.0     NaN   -68.0   351.0   518.0   441.0   
2019-01-22 00:03:14         187.0     NaN  -113.0   306.0   494.0   396.0   
2019-01-22 00:04:14           NaN     NaN     NaN     NaN     NaN     NaN   
2019-01-22 00:05:14          22.0     NaN   486.0   186.0   396.0   276.0   
2019-01-22 00:06:14         -49.0     NaN   425.0   111.0   335.0   201.0   
2019-01-22 00:07:14         654.0     NaN   354.0    51.0   264.0   141.0   
2019-01-22 00:08:14         609.0     NaN   309.0     6.0   219.0    96.0   
2019-01-22 00:09:14         557.0     NaN   257.0   -54.0   167.0    36.0   
2019-01-22 00:10:14         495.0     NaN   195.0    69.0   105.0   159.0   
2019-01-22 00:11:14         457.0     NaN   157.0   -10.0    67.0    80.0   
2019-01-22 00:12:14         375.0     NaN    75.0   906.0   -15.0    16.0   
2019-01-22 00:13:14         283.0     NaN   -17.0   846.0   410.0   936.0   
2019-01-22 00:14:14          56.0     NaN   438.0   774.0   348.0   864.0   
2019-01-22 00:15:14          -2.0     NaN   397.0   726.0   307.0   816.0   
2019-01-22 00:16:14         -62.0     NaN   342.0   666.0   252.0   756.0   
2019-01-22 00:17:14         552.0     NaN   252.0   591.0   162.0   681.0   
2019-01-22 00:18:14         489.0     NaN   189.0   534.0    99.0   624.0   
2019-01-22 00:19:14         458.0     NaN   158.0   486.0    68.0   576.0   
2019-01-22 00:20:14         376.0     NaN    76.0   426.0   -14.0   516.0   
2019-01-22 00:21:14         264.0     NaN   -36.0   351.0   487.0   441.0   
2019-01-22 00:22:14          85.0     NaN   516.0   306.0   426.0   396.0   
2019-01-22 00:23:14          28.0     NaN   489.0   249.0   399.0   339.0   
2019-01-22 00:24:14         -31.0     NaN   433.0   186.0   343.0   276.0   
2019-01-22 00:25:14         664.0     NaN   364.0   126.0   274.0   216.0   
2019-01-22 00:26:14         590.0     NaN   290.0    66.0   200.0   156.0   
2019-01-22 00:27:14         536.0     NaN   236.0    32.0   146.0   122.0   
2019-01-22 00:28:14         479.0     NaN   179.0   -25.0    89.0    65.0   
2019-01-22 00:29:14         392.0     NaN    92.0  -114.0     2.0   -24.0   
...                           ...     ...     ...     ...     ...     ...   
2019-01-23 23:30:14         -45.0     NaN   201.0   605.0   111.0   815.0   
2019-01-23 23:31:14         458.0     NaN   158.0   546.0    68.0   756.0   
2019-01-23 23:32:14         373.0     NaN    73.0   485.0   -17.0   695.0   
2019-01-23 23:33:14         288.0     NaN   -12.0   425.0    84.0   635.0   
2019-01-23 23:34:14         228.0     NaN   -72.0   365.0    15.0   575.0   
2019-01-23 23:35:14          60.0     NaN    71.0   305.0   168.0   515.0   
2019-01-23 23:36:14          -7.0     NaN    -7.0   231.0   109.0   441.0   
2019-01-23 23:37:14         -49.0     NaN   -68.0   170.0    62.0   380.0   
2019-01-23 23:38:14         187.0     NaN  -113.0   125.0    17.0   335.0   
2019-01-23 23:39:14          57.0     NaN    47.0    66.0   -43.0   276.0   
2019-01-23 23:40:14          -2.0     NaN    91.0   -27.0     1.0   183.0   
2019-01-23 23:41:14         -38.0     NaN    77.0   785.0   -13.0    29.0   
2019-01-23 23:42:14         309.0     NaN     9.0   725.0    44.0   815.0   
2019-01-23 23:43:14          75.0     NaN    57.0   650.0   -33.0   740.0   
2019-01-23 23:44:14           5.0     NaN    57.0   591.0   679.0   681.0   
2019-01-23 23:45:14         -59.0     NaN   -48.0   531.0   608.0   621.0   
2019-01-23 23:46:14          87.0     NaN   632.0   486.0   542.0   576.0   
2019-01-23 23:47:14          26.0     NaN   593.0   425.0   503.0   515.0   
2019-01-23 23:48

OK, now can I flatten that header? (Found answer [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.cat.html))

In [65]:
cols = df2.columns.get_level_values(1).str.cat(df2.columns.get_level_values(0), sep="_")
cols


Index(['1_101_N_timeToArrival', '1_101_S_timeToArrival',
       '1_103_N_timeToArrival', '1_103_S_timeToArrival',
       '1_104_N_timeToArrival', '1_104_S_timeToArrival',
       '1_106_N_timeToArrival', '1_106_S_timeToArrival',
       '1_107_N_timeToArrival', '1_107_S_timeToArrival',
       ...
       '1_136_N_timeToDeparture', '1_136_S_timeToDeparture',
       '1_137_N_timeToDeparture', '1_137_S_timeToDeparture',
       '1_138_N_timeToDeparture', '1_138_S_timeToDeparture',
       '1_139_N_timeToDeparture', '1_139_S_timeToDeparture',
       '1_142_N_timeToDeparture', '1_142_S_timeToDeparture'],
      dtype='object', name='name', length=152)

In [68]:
df2.columns = cols
df2

name,1_101_N_timeToArrival,1_101_S_timeToArrival,1_103_N_timeToArrival,1_103_S_timeToArrival,1_104_N_timeToArrival,1_104_S_timeToArrival,1_106_N_timeToArrival,1_106_S_timeToArrival,1_107_N_timeToArrival,1_107_S_timeToArrival,...,1_136_N_timeToDeparture,1_136_S_timeToDeparture,1_137_N_timeToDeparture,1_137_S_timeToDeparture,1_138_N_timeToDeparture,1_138_S_timeToDeparture,1_139_N_timeToDeparture,1_139_S_timeToDeparture,1_142_N_timeToDeparture,1_142_S_timeToDeparture
snapshotNYC,,,,,,,,,,,,,,,,,,,,,
2019-01-22 00:00:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247.0,256.0,157.0,436.0,97.0,526.0,37.0,616.0,546.0,NaN
2019-01-22 00:01:14,358.0,NaN,58.0,426.0,621.0,516.0,531.0,606.0,441.0,696.0,...,193.0,173.0,103.0,353.0,43.0,443.0,636.0,533.0,486.0,NaN
2019-01-22 00:02:14,232.0,NaN,-68.0,351.0,518.0,441.0,428.0,531.0,338.0,621.0,...,123.0,60.0,33.0,240.0,621.0,330.0,561.0,420.0,411.0,NaN
2019-01-22 00:03:14,187.0,NaN,-113.0,306.0,494.0,396.0,404.0,486.0,314.0,576.0,...,79.0,13.0,624.0,193.0,564.0,283.0,504.0,373.0,354.0,NaN
2019-01-22 00:04:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.0,393.0,576.0,163.0,516.0,253.0,456.0,343.0,306.0,NaN
2019-01-22 00:05:14,22.0,NaN,486.0,186.0,396.0,276.0,306.0,366.0,216.0,456.0,...,606.0,333.0,516.0,74.0,456.0,164.0,396.0,254.0,246.0,NaN
2019-01-22 00:06:14,-49.0,NaN,425.0,111.0,335.0,201.0,245.0,291.0,155.0,381.0,...,546.0,273.0,456.0,453.0,396.0,6.0,336.0,96.0,186.0,NaN
2019-01-22 00:07:14,654.0,NaN,354.0,51.0,264.0,141.0,174.0,231.0,84.0,321.0,...,471.0,613.0,381.0,793.0,321.0,883.0,261.0,10.0,111.0,NaN
2019-01-22 00:08:14,609.0,NaN,309.0,6.0,219.0,96.0,129.0,186.0,39.0,276.0,...,426.0,590.0,336.0,770.0,276.0,860.0,216.0,-35.0,66.0,NaN


In [69]:
df2.reset_index(inplace=True)
df2

name,snapshotNYC,1_101_N_timeToArrival,1_101_S_timeToArrival,1_103_N_timeToArrival,1_103_S_timeToArrival,1_104_N_timeToArrival,1_104_S_timeToArrival,1_106_N_timeToArrival,1_106_S_timeToArrival,1_107_N_timeToArrival,...,1_136_N_timeToDeparture,1_136_S_timeToDeparture,1_137_N_timeToDeparture,1_137_S_timeToDeparture,1_138_N_timeToDeparture,1_138_S_timeToDeparture,1_139_N_timeToDeparture,1_139_S_timeToDeparture,1_142_N_timeToDeparture,1_142_S_timeToDeparture
0,2019-01-22 00:00:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,247.0,256.0,157.0,436.0,97.0,526.0,37.0,616.0,546.0,NaN
1,2019-01-22 00:01:14,358.0,NaN,58.0,426.0,621.0,516.0,531.0,606.0,441.0,...,193.0,173.0,103.0,353.0,43.0,443.0,636.0,533.0,486.0,NaN
2,2019-01-22 00:02:14,232.0,NaN,-68.0,351.0,518.0,441.0,428.0,531.0,338.0,...,123.0,60.0,33.0,240.0,621.0,330.0,561.0,420.0,411.0,NaN
3,2019-01-22 00:03:14,187.0,NaN,-113.0,306.0,494.0,396.0,404.0,486.0,314.0,...,79.0,13.0,624.0,193.0,564.0,283.0,504.0,373.0,354.0,NaN
4,2019-01-22 00:04:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.0,393.0,576.0,163.0,516.0,253.0,456.0,343.0,306.0,NaN
5,2019-01-22 00:05:14,22.0,NaN,486.0,186.0,396.0,276.0,306.0,366.0,216.0,...,606.0,333.0,516.0,74.0,456.0,164.0,396.0,254.0,246.0,NaN
6,2019-01-22 00:06:14,-49.0,NaN,425.0,111.0,335.0,201.0,245.0,291.0,155.0,...,546.0,273.0,456.0,453.0,396.0,6.0,336.0,96.0,186.0,NaN
7,2019-01-22 00:07:14,654.0,NaN,354.0,51.0,264.0,141.0,174.0,231.0,84.0,...,471.0,613.0,381.0,793.0,321.0,883.0,261.0,10.0,111.0,NaN
8,2019-01-22 00:08:14,609.0,NaN,309.0,6.0,219.0,96.0,129.0,186.0,39.0,...,426.0,590.0,336.0,770.0,276.0,860.0,216.0,-35.0,66.0,NaN
9,2019-01-22 00:09:14,557.0,NaN,257.0,-54.0,167.0,36.0,77.0,126.0,530.0,...,366.0,531.0,276.0,711.0,216.0,801.0,156.0,-95.0,6.0,NaN


In [60]:
df2.shape

(2855, 153)

## Marking "in station" events

My operating theory here is that for a given route_station_direction (a train platform, really), that if the current time to next train is _greater_ than the last reading (usually a minute ago) then a train just left the station. So:



Ooooh: We can calculate the time since the train last left the station.

In [12]:
df = pickle.load(open('saved_sample3.p', 'rb'))

In [41]:
# This whole block calculates the gaps for each route-station-direction entry

# for this to work, we have to be sorted by the entry name and then by datetime
df.sort_values(['name', 'snapshotUnix'])

# establish initial settings
last_seconds_to_arrival = 0
last_name = ""
last_in_station_time = 1548133274   # <- eventually this could be stored in a db for next data check
left_station_array = []
gap = []

# run through all of the rows using zip, which is superfast
for name, seconds_to_arrival, snapshot_time in zip(df.name.values, df.timeToArrival.values, df.snapshotUnix.values):

    left_station = 0
    
    if name != last_name:
        # we've hit a new route/station/direction name
        # reset the 'last' values
        last_seconds_to_arrival = 0
        last_in_station_time = 1548133274
        last_name = name
        

    if seconds_to_arrival > last_seconds_to_arrival:
        # train must have just left the station
        # reset last in station to now
        left_station = 1
        last_in_station_time = snapshot_time
    
    # no matter what, calculate the time since the last time in the station
    seconds_since_in_station = (snapshot_time - last_in_station_time)
    
    # calculate the current gap and add it to the array
    gap.append(seconds_since_in_station + seconds_to_arrival)
    
    # calculate the current in_station and add it to the array
    left_station_array.append(left_station)
    
    # save the current time to arrival for next use
    last_seconds_to_arrival = seconds_to_arrival
    
df['left_station'] = left_station_array
df['gap'] = gap
        

In [33]:
df

,id,snapshotUnix,snapshotNYC,routeId,stationIdGTFS,direction,trainOrderLine,trainOrderAll,arrivalTime,departureTime,updatedOn,timeToArrival,timeToDeparture,delay,name,left_station,gap
0,128769,1548133274,2019-01-22 00:01:14,1,101,N,0,0,1.548134e+09,NaN,1548133254,358.0,NaN,None,1_101_N,1,358.0
1,133212,1548133334,2019-01-22 00:02:14,1,101,N,0,0,1.548134e+09,NaN,1548133329,232.0,NaN,None,1_101_N,0,292.0
2,137594,1548133394,2019-01-22 00:03:14,1,101,N,0,0,1.548134e+09,NaN,1548133374,187.0,NaN,None,1_101_N,0,307.0
3,145548,1548133514,2019-01-22 00:05:14,1,101,N,0,0,1.548134e+09,NaN,1548133494,22.0,NaN,None,1_101_N,0,262.0
4,149848,1548133574,2019-01-22 00:06:14,1,101,N,0,0,1.548134e+09,NaN,1548133569,-49.0,NaN,None,1_101_N,0,251.0
5,154190,1548133634,2019-01-22 00:07:14,1,101,N,0,0,1.548134e+09,NaN,1548133629,654.0,NaN,None,1_101_N,1,654.0
6,158503,1548133694,2019-01-22 00:08:14,1,101,N,0,0,1.548134e+09,NaN,1548133674,609.0,NaN,None,1_101_N,0,669.0
7,162820,1548133754,2019-01-22 00:09:14,1,101,N,0,0,1.548134e+09,NaN,1548133734,557.0,NaN,None,1_101_N,0,677.0
8,167092,1548133814,2019-01-22 00:10:14,1,101,N,0,0,1.548134e+09,NaN,1548133794,495.0,NaN,None,1_101_N,0,675.0
9,171316,1548133874,2019-01-22 00:11:14,1,101,N,0,0,1.548134e+09,NaN,1548133854,457.0,NaN,None,1_101_N,0,697.0


## Making a Gap Chart

In [43]:
df_gap = df.pivot(index='snapshotNYC', columns='name', values='gap')

In [44]:
df_gap

name,1_101_N,1_101_S,1_103_N,1_103_S,1_104_N,1_104_S,1_106_N,1_106_S,1_107_N,1_107_S,1_108_N,1_108_S,1_109_N,1_109_S,1_110_N,1_110_S,1_111_N,1_111_S,1_112_N,1_112_S,1_113_N,1_113_S,1_114_N,1_114_S,1_115_N,1_115_S,1_116_N,1_116_S,1_117_N,1_117_S,1_118_N,1_118_S,1_119_N,1_119_S,1_120_N,1_120_S,1_121_N,1_121_S,1_122_N,1_122_S,1_123_N,1_123_S,1_124_N,1_124_S,1_125_N,1_125_S,1_126_N,1_126_S,1_127_N,1_127_S,1_128_N,1_128_S,1_129_N,1_129_S,1_130_N,1_130_S,1_131_N,1_131_S,1_132_N,1_132_S,1_133_N,1_133_S,1_134_N,1_134_S,1_135_N,1_135_S,1_136_N,1_136_S,1_137_N,1_137_S,1_138_N,1_138_S,1_139_N,1_139_S,1_142_N,1_142_S
snapshotNYC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-01-22 00:00:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,577.0,273.0,487.0,363.0,397.0,76.0,307.0,196.0,247.0,256.0,157.0,316.0,97.0,526.0,37.0,616.0,NaN,108.0
2019-01-22 00:01:14,358.0,NaN,58.0,426.0,621.0,516.0,531.0,606.0,441.0,696.0,351.0,756.0,261.0,51.0,201.0,141.0,111.0,231.0,597.0,351.0,477.0,441.0,357.0,561.0,267.0,-228.0,147.0,-138.0,27.0,12.0,449.0,72.0,359.0,132.0,269.0,252.0,179.0,372.0,89.0,432.0,617.0,522.0,557.0,612.0,467.0,702.0,347.0,822.0,227.0,-117.0,137.0,-27.0,47.0,33.0,-13.0,93.0,583.0,153.0,583.0,273.0,493.0,363.0,403.0,53.0,313.0,173.0,253.0,233.0,163.0,293.0,103.0,503.0,636.0,593.0,NaN,58.0
2019-01-22 00:02:14,292.0,NaN,-8.0,411.0,578.0,501.0,488.0,591.0,398.0,681.0,308.0,741.0,218.0,771.0,158.0,126.0,68.0,216.0,594.0,336.0,474.0,426.0,354.0,546.0,264.0,-403.0,144.0,-313.0,415.0,-163.0,415.0,-103.0,325.0,-43.0,235.0,77.0,145.0,197.0,55.0,257.0,619.0,347.0,559.0,437.0,469.0,527.0,349.0,647.0,229.0,-132.0,139.0,-42.0,633.0,18.0,573.0,78.0,573.0,138.0,573.0,258.0,483.0,348.0,393.0,318.0,303.0,120.0,243.0,180.0,153.0,240.0,621.0,450.0,621.0,540.0,NaN,540.0
2019-01-22 00:03:14,307.0,NaN,7.0,426.0,614.0,516.0,524.0,606.0,434.0,696.0,344.0,756.0,254.0,786.0,194.0,121.0,574.0,211.0,574.0,331.0,454.0,421.0,334.0,541.0,244.0,511.0,124.0,601.0,414.0,751.0,414.0,811.0,324.0,871.0,234.0,991.0,144.0,238.0,571.0,298.0,601.0,388.0,541.0,478.0,451.0,568.0,331.0,688.0,211.0,-117.0,121.0,-27.0,650.0,33.0,590.0,93.0,590.0,153.0,589.0,261.0,499.0,351.0,409.0,321.0,319.0,381.0,259.0,193.0,624.0,253.0,624.0,463.0,624.0,553.0,NaN,553.0
2019-01-22 00:04:14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,567.0,273.0,477.0,363.0,387.0,333.0,297.0,393.0,237.0,393.0,636.0,283.0,636.0,493.0,636.0,583.0,NaN,583.0
2019-01-22 00:05:14,262.0,NaN,486.0,426.0,636.0,516.0,546.0,606.0,456.0,696.0,366.0,756.0,276.0,786.0,538.0,696.0,568.0,786.0,568.0,351.0,448.0,441.0,328.0,561.0,464.0,531.0,344.0,621.0,404.0,771.0,404.0,831.0,314.0,891.0,224.0,1011.0,553.0,1011.0,583.0,276.0,613.0,366.0,553.0,456.0,463.0,546.0,343.0,666.0,628.0,-117.0,538.0,-27.0,628.0,33.0,568.0,93.0,568.0,153.0,568.0,273.0,478.0,363.0,388.0,333.0,666.0,393.0,606.0,393.0,636.0,254.0,636.0,464.0,636.0,554.0,NaN,554.0
2019-01-22 00:06:14,251.0,NaN,485.0,411.0,635.0,501.0,545.0,591.0,455.0,681.0,365.0,741.0,504.0,771.0,504.0,681.0,534.0,771.0,534.0,317.0,414.0,407.0,294.0,527.0,449.0,497.0,329.0,587.0,389.0,737.0,389.0,797.0,299.0,857.0,209.0,977.0,518.0,977.0,548.0,977.0,578.0,360.0,518.0,450.0,428.0,540.0,308.0,660.0,594.0,450.0,504.0,82.0,594.0,142.0,534.0,202.0,534.0,262.0,549.0,273.0,459.0,363.0,369.0,333.0,666.0,393.0,606.0,393.0,636.0,333.0,636.0,366.0,636.0,456.0,NaN,456.0
2019-01-22 00:07:14,654.0,NaN,474.0,411.0,624.0,501.0,534.0,591.0,444.0,681.0,354.0,741.0,515.0,771.0,515.0,681.0,545.0,771.0,545.0,771.0,425.0,376.0,419.0,496.0,449.0,466.0,329.0,556.0,389.0,706.0,389.0,766.0,299.0,826.0,209.0,946.0,477.

In [ ]:
df.to_csv('')